In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better data viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Environment setup complete!")

Environment setup complete!


In [2]:
# Load the dataset
df = pd.read_csv('uber.csv')

print("=== DATASET BASIC INFORMATION ===")
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]}")

print("\n=== FIRST 5 ROWS ===")
print(df.head())

print("\n=== LAST 5 ROWS ===")
print(df.tail())

print("\n=== COLUMN INFORMATION ===")
print(df.info())

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== COLUMN NAMES ===")
print(df.columns.tolist())

=== DATASET BASIC INFORMATION ===
Dataset shape: (200000, 9)
Number of rows: 200,000
Number of columns: 9

=== FIRST 5 ROWS ===
   Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1 

In [3]:
print("=== DATA QUALITY ASSESSMENT ===")

# Check for missing values
print("\n--- Missing Values Analysis ---")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percentage': missing_percentage.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found!")

# Check for duplicate rows
print(f"\n--- Duplicate Analysis ---")
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count:,}")
print(f"Percentage of duplicates: {(duplicate_count/len(df)*100):.2f}%")

# Check for duplicate keys (should be unique identifiers)
duplicate_keys = df['key'].duplicated().sum()
print(f"Number of duplicate keys: {duplicate_keys:,}")

# Basic statistical summary
print("\n--- Statistical Summary ---")
print(df.describe())

# Check for zero coordinates (invalid locations)
zero_coords = df[(df['pickup_longitude'] == 0) | (df['pickup_latitude'] == 0) |
                 (df['dropoff_longitude'] == 0) | (df['dropoff_latitude'] == 0)]
print(f"\nRows with zero coordinates: {len(zero_coords):,}")
print(f"Percentage with zero coordinates: {(len(zero_coords)/len(df)*100):.2f}%")

=== DATA QUALITY ASSESSMENT ===

--- Missing Values Analysis ---
              Column  Missing_Count  Missing_Percentage
6  dropoff_longitude              1              0.0005
7   dropoff_latitude              1              0.0005

--- Duplicate Analysis ---
Number of duplicate rows: 0
Percentage of duplicates: 0.00%
Number of duplicate keys: 0

--- Statistical Summary ---
         Unnamed: 0    fare_amount  pickup_longitude  pickup_latitude  \
count  2.000000e+05  200000.000000     200000.000000    200000.000000   
mean   2.771250e+07      11.359955        -72.527638        39.935885   
std    1.601382e+07       9.901776         11.437787         7.720539   
min    1.000000e+00     -52.000000      -1340.648410       -74.015515   
25%    1.382535e+07       6.000000        -73.992065        40.734796   
50%    2.774550e+07       8.500000        -73.981823        40.752592   
75%    4.155530e+07      12.500000        -73.967154        40.767158   
max    5.542357e+07     499.000000    

In [4]:
print("=== DATA TYPE CONVERSIONS ===")

# Convert pickup_datetime to datetime
print("Converting pickup_datetime to datetime format...")
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])

# Verify the conversion
print(f"Datetime conversion successful: {df["pickup_datetime"].dtype}")
print(f"Date range: {df["pickup_datetime"].min()} to {df["pickup_datetime"].max()}")

# Check for any invalid passenger counts
print(f"\n--- Passenger Count Analysis ---")
print(f"Unique passenger counts: {sorted(df["passenger_count"].unique())}")
print(f"Passenger count value counts:")
print(df["passenger_count"].value_counts().sort_index())

# Check for negative or zero fare amounts
print(f"\n--- Fare Amount Analysis ---")
negative_fares = df[df["fare_amount"] <= 0]
print(f"Rides with negative or zero fare: {len(negative_fares):,}")
print(f"Percentage: {(len(negative_fares)/len(df)*100):.2f}%")
if len(negative_fares) > 0:
    print("Sample of negative/zero fares:")
    print(negative_fares[["key", "fare_amount", "pickup_datetime"]].head())

=== DATA TYPE CONVERSIONS ===
Converting pickup_datetime to datetime format...
Datetime conversion successful: datetime64[ns, UTC]
Date range: 2009-01-01 01:15:22+00:00 to 2015-06-30 23:40:39+00:00

--- Passenger Count Analysis ---
Unique passenger counts: [0, 1, 2, 3, 4, 5, 6, 208]
Passenger count value counts:
passenger_count
0         709
1      138425
2       29428
3        8881
4        4276
5       14009
6        4271
208         1
Name: count, dtype: int64

--- Fare Amount Analysis ---
Rides with negative or zero fare: 22
Percentage: 0.01%
Sample of negative/zero fares:
                               key  fare_amount           pickup_datetime
20744  2015-04-22 23:25:07.0000008          0.0 2015-04-22 23:25:07+00:00
22182  2010-03-20 02:59:51.0000002          0.0 2010-03-20 02:59:51+00:00
63395  2015-03-03 23:07:41.0000008         -5.0 2015-03-03 23:07:41+00:00
71246  2010-02-11 21:47:10.0000001         -3.3 2010-02-11 21:47:10+00:00
79903  2015-05-01 14:43:02.0000004         -3.

In [5]:
print("=== DATA CLEANING PROCESS ===")

# Store original dataset size
original_size = len(df)
print(f"Original dataset size: {original_size:,} rows")

# Step 5.1: Remove duplicate rows
df_cleaned = df.drop_duplicates()
after_duplicates = len(df_cleaned)
print(f"After removing duplicates: {after_duplicates:,} rows ({original_size - after_duplicates:,} removed)")

# Step 5.2: Remove rows with zero coordinates (invalid locations)
df_cleaned = df_cleaned[
    (df_cleaned["pickup_longitude"] != 0) &
    (df_cleaned["pickup_latitude"] != 0) &
    (df_cleaned["dropoff_longitude"] != 0) &
    (df_cleaned["dropoff_latitude"] != 0)
]
after_coordinates = len(df_cleaned)
print(f"After removing zero coordinates: {after_coordinates:,} rows ({after_duplicates - after_coordinates:,} removed)")

# Step 5.3: Remove negative or zero fare amounts
df_cleaned = df_cleaned[df_cleaned["fare_amount"] > 0]
after_fares = len(df_cleaned)
print(f"After removing invalid fares: {after_fares:,} rows ({after_coordinates - after_fares:,} removed)")

# Step 5.4: Remove invalid passenger counts (0 or negative)
df_cleaned = df_cleaned[df_cleaned["passenger_count"] > 0]
after_passengers = len(df_cleaned)
print(f"After removing invalid passenger counts: {after_passengers:,} rows ({after_fares - after_passengers:,} removed)")

# Step 5.5: Remove extreme outliers in fare amounts (optional - be careful with this)
# Calculate IQR for fare amounts
Q1 = df_cleaned["fare_amount"].quantile(0.25)
Q3 = df_cleaned["fare_amount"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR # Using 3*IQR instead of 1.5*IQR for less aggressive outlier removal
upper_bound = Q3 + 3 * IQR

print(f"\nFare amount outlier bounds:")
print(f"Lower bound: ${lower_bound:.2f}")
print(f"Upper bound: ${upper_bound:.2f}")

# Count extreme outliers
extreme_outliers = df_cleaned[(df_cleaned["fare_amount"] < lower_bound) |
                              (df_cleaned["fare_amount"] > upper_bound)]
print(f"Extreme fare outliers found: {len(extreme_outliers):,}")

# Optionally remove extreme outliers (uncomment if needed)
# df_cleaned = df_cleaned[(df_cleaned["fare_amount"] >= lower_bound) &
#                         (df_cleaned["fare_amount"] <= upper_bound)]
# after_outliers = len(df_cleaned)
# print(f"After removing extreme outliers: {after_outliers:,} rows")

# Final cleaning summary

final_size = len(df_cleaned)
total_removed = original_size - final_size
removal_percentage = (total_removed / original_size) * 100

print(f"\n=== CLEANING SUMMARY ===")
print(f"Original size: {original_size:,} rows")
print(f"Final size: {final_size:,} rows")
print(f"Total removed: {total_removed:,} rows ({removal_percentage:.2f}%)")
print(f"Data retention rate: {((final_size/original_size)*100):.2f}%")

# Verify cleaned data
print(f"\n=== CLEANED DATA VERIFICATION ===")
print(f"No missing values: {df_cleaned.isnull().sum().sum() == 0}")
print(f"No zero coordinates: {((df_cleaned[["pickup_longitude",
"pickup_latitude", "dropoff_longitude", "dropoff_latitude"]] == 0).sum().sum()
== 0)}")
print(f"All positive fares: {(df_cleaned["fare_amount"] > 0).all()}")
print(f"All positive passenger counts: {(df_cleaned["passenger_count"] >
0).all()}")

=== DATA CLEANING PROCESS ===
Original dataset size: 200,000 rows
After removing duplicates: 200,000 rows (0 removed)
After removing zero coordinates: 196,032 rows (3,968 removed)
After removing invalid fares: 196,013 rows (19 removed)
After removing invalid passenger counts: 195,326 rows (687 removed)

Fare amount outlier bounds:
Lower bound: $-13.50
Upper bound: $32.00
Extreme fare outliers found: 8,688

=== CLEANING SUMMARY ===
Original size: 200,000 rows
Final size: 195,326 rows
Total removed: 4,674 rows (2.34%)
Data retention rate: 97.66%

=== CLEANED DATA VERIFICATION ===
No missing values: True
No zero coordinates: True
All positive fares: True
All positive passenger counts: True


In [ ]:
# 6 Save cleaned dataset
output_filename = "uber_cleaned.csv"
df_cleaned.to_csv(output_filename, index=False)

print(f"\nCleaned dataset saved as: {output_filename}")
print(f"File size: {len(df_cleaned):,} rows × {len(df_cleaned.columns)} columns")

# Display final dataset info
print("\n=== FINAL CLEANED DATASET INFO ===")
print(df_cleaned.info())
print("\nFirst 5 rows of cleaned data:")
print(df_cleaned.head())


Cleaned dataset saved as: uber_cleaned.csv
File size: 195,326 rows × 9 columns

=== FINAL CLEANED DATASET INFO ===
<class 'pandas.core.frame.DataFrame'>
Index: 195326 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   Unnamed: 0         195326 non-null  int64              
 1   key                195326 non-null  object             
 2   fare_amount        195326 non-null  float64            
 3   pickup_datetime    195326 non-null  datetime64[ns, UTC]
 4   pickup_longitude   195326 non-null  float64            
 5   pickup_latitude    195326 non-null  float64            
 6   dropoff_longitude  195326 non-null  float64            
 7   dropoff_latitude   195326 non-null  float64            
 8   passenger_count    195326 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(2), object(1)
memory usage: 14.9+ MB
None

First 5 rows of

In [ ]:
print("=== COMPREHENSIVE DESCRIPTIVE STATISTICS ===")

# Basic descriptive statistics for numerical columns
numerical_cols = ["fare_amount", "pickup_longitude", "pickup_latitude",
                  "dropoff_longitude", "dropoff_latitude", "passenger_count"]
print("--- Basic Statistical Measures ---")
desc_stats = df_cleaned[numerical_cols].describe()
print(desc_stats)

# Additional statistical measures
print("\n--- Additional Statistical Measures ---")
for col in numerical_cols:
    data = df_cleaned[col]
    print(f"\n{col.upper()}:")
    print(f" Mean: {data.mean():.4f}")
    print(f" Median: {data.median():.4f}")
    print(f" Mode: {data.mode().iloc[0]:.4f}")
    print(f" Standard Deviation: {data.std():.4f}")
    print(f" Variance: {data.var():.4f}")
    print(f" Skewness: {data.skew():.4f}")
    print(f" Kurtosis: {data.kurtosis():.4f}")
    print(f" Range: {data.max() - data.min():.4f}")
    print(f" IQR: {data.quantile(0.75) - data.quantile(0.25):.4f}")

# Quartile analysis
print("\n--- Quartile Analysis ---")
quartiles = df_cleaned[numerical_cols].quantile([0.25, 0.5, 0.75])
print(quartiles)

# Percentile analysis for fare amounts
print("\n--- Fare Amount Percentile Analysis ---")
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
fare_percentiles = df_cleaned["fare_amount"].quantile([p/100 for p in percentiles])
for i, p in enumerate(percentiles):
    print(f"{p}th percentile: ${fare_percentiles.iloc[i]:.2f}")

In [ ]:
print("=== UNIVARIATE ANALYSIS ===")

# Set up the plotting environment
plt.style.use("default")
fig = plt.figure(figsize=(20, 15))

# 8.1: Fare Amount Distribution
plt.subplot(3, 3, 1)
plt.hist(df_cleaned["fare_amount"], bins=50, alpha=0.7, color="skyblue",
         edgecolor="black")
plt.title("Distribution of Fare Amounts", fontsize=14, fontweight="bold")
plt.xlabel("Fare Amount ($)")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)
# Add statistics to the plot
mean_fare = df_cleaned["fare_amount"].mean()
median_fare = df_cleaned["fare_amount"].median()
plt.axvline(mean_fare, color="red", linestyle="--", label=f"Mean:\n${mean_fare:.2f}")
plt.axvline(median_fare, color="green", linestyle="--", label=f"Median:\n${median_fare:.2f}")
plt.legend()

# 8.2: Fare Amount Box Plot
plt.subplot(3, 3, 2)
box_plot = plt.boxplot(df_cleaned["fare_amount"], patch_artist=True)
box_plot["boxes"][0].set_facecolor("lightblue")
plt.title("Fare Amount Box Plot", fontsize=14, fontweight="bold")
plt.ylabel("Fare Amount ($)")
plt.grid(True, alpha=0.3)

# 8.3: Passenger Count Distribution
plt.subplot(3, 3, 3)
passenger_counts = df_cleaned["passenger_count"].value_counts().sort_index()
plt.bar(passenger_counts.index, passenger_counts.values, color="lightcoral",
        alpha=0.7)
plt.title("Distribution of Passenger Counts", fontsize=14, fontweight="bold")
plt.xlabel("Number of Passengers")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)
# Add percentage labels on bars
total_rides = len(df_cleaned)
for i, v in enumerate(passenger_counts.values):
    plt.text(passenger_counts.index[i], v + total_rides*0.01,
             f"{(v/total_rides*100):.1f}%", ha="center", va="bottom")

# 8.4: Pickup Longitude Distribution
plt.subplot(3, 3, 4)
plt.hist(df_cleaned["pickup_longitude"], bins=50, alpha=0.7,
         color="lightgreen", edgecolor="black")
plt.title("Distribution of Pickup Longitude", fontsize=14, fontweight="bold")
plt.xlabel("Longitude")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)

# 8.5: Pickup Latitude Distribution
plt.subplot(3, 3, 5)
plt.hist(df_cleaned["pickup_latitude"], bins=50, alpha=0.7, color="orange",
         edgecolor="black")
plt.title("Distribution of Pickup Latitude", fontsize=14, fontweight="bold")
plt.xlabel("Latitude")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)

# 8.6: Dropoff Longitude Distribution
plt.subplot(3, 3, 6)
plt.hist(df_cleaned["dropoff_longitude"], bins=50, alpha=0.7, color="purple",
         edgecolor="black")
plt.title("Distribution of Dropoff Longitude", fontsize=14, fontweight="bold")
plt.xlabel("Longitude")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)

# 8.7: Dropoff Latitude Distribution
plt.subplot(3, 3, 7)
plt.hist(df_cleaned["dropoff_latitude"], bins=50, alpha=0.7, color="brown",
         edgecolor="black")
plt.title("Distribution of Dropoff Latitude", fontsize=14, fontweight="bold")
plt.xlabel("Latitude")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)

# 8.8: Log-transformed Fare Amount (to handle skewness)
plt.subplot(3, 3, 8)
log_fares = np.log1p(df_cleaned["fare_amount"]) # log1p handles zero values better
plt.hist(log_fares, bins=50, alpha=0.7, color="pink", edgecolor="black")
plt.title("Log-Transformed Fare Amounts", fontsize=14, fontweight="bold")
plt.xlabel("Log(Fare Amount + 1)")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)

# 8.9: Fare Amount vs Passenger Count (Box Plot)
plt.subplot(3, 3, 9)
df_cleaned.boxplot(column="fare_amount", by="passenger_count", ax=plt.gca())
plt.title("Fare Amount by Passenger Count", fontsize=14, fontweight="bold")
plt.xlabel("Passenger Count")
plt.ylabel("Fare Amount ($)")
plt.suptitle("") # Remove the automatic title
plt.tight_layout()
plt.savefig("univariate_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

# Print summary statistics for univariate analysis
print("\n--- Univariate Analysis Summary ---")
print(f"Fare Amount - Mean: ${df_cleaned['fare_amount'].mean():.2f}, Std:\n${df_cleaned['fare_amount'].std():.2f}")
print(f"Most common passenger count:\n{df_cleaned['passenger_count'].mode().iloc[0]} passengers")
print(f"Longitude range: {df_cleaned['pickup_longitude'].min():.4f} to\n{df_cleaned['pickup_longitude'].max():.4f}")
print(f"Latitude range: {df_cleaned['pickup_latitude'].min():.4f} to\n{df_cleaned['pickup_latitude'].max():.4f}") 

In [ ]:
print("=== BIVARIATE ANALYSIS ===")

# 9.1: Correlation Matrix
print("--- Correlation Analysis ---")
correlation_matrix = df_cleaned[numerical_cols].corr()
print("Correlation Matrix:")
print(correlation_matrix.round(4))

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool)) # Mask upper triangle
sns.heatmap(correlation_matrix,
            mask=mask,
            annot=True,
            cmap="coolwarm",
            center=0,
            square=True,
            fmt=".3f",
            cbar_kws={"shrink": .8})
plt.title("Correlation Matrix Heatmap", fontsize=16, fontweight="bold", pad=20)
plt.tight_layout()
plt.savefig("correlation_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()

# 9.2: Scatter Plot Analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Fare vs Pickup Longitude
axes[0, 0].scatter(df_cleaned["pickup_longitude"], df_cleaned["fare_amount"],
                   alpha=0.5, s=1, color="blue")
axes[0, 0].set_title("Fare Amount vs Pickup Longitude")
axes[0, 0].set_xlabel("Pickup Longitude")
axes[0, 0].set_ylabel("Fare Amount ($)")
axes[0, 0].grid(True, alpha=0.3)

# Fare vs Pickup Latitude
axes[0, 1].scatter(df_cleaned["pickup_latitude"], df_cleaned["fare_amount"],
                   alpha=0.5, s=1, color="green")
axes[0, 1].set_title("Fare Amount vs Pickup Latitude")
axes[0, 1].set_xlabel("Pickup Latitude")
axes[0, 1].set_ylabel("Fare Amount ($)")
axes[0, 1].grid(True, alpha=0.3)

# Fare vs Passenger Count
axes[0, 2].scatter(df_cleaned["passenger_count"], df_cleaned["fare_amount"],
                   alpha=0.5, s=1, color="red")
axes[0, 2].set_title("Fare Amount vs Passenger Count")
axes[0, 2].set_xlabel("Passenger Count")
axes[0, 2].set_ylabel("Fare Amount ($)")
axes[0, 2].grid(True, alpha=0.3)

# Pickup vs Dropoff Longitude
axes[1, 0].scatter(df_cleaned["pickup_longitude"],
                   df_cleaned["dropoff_longitude"],
                   alpha=0.5, s=1, color="purple")
axes[1, 0].set_title("Pickup vs Dropoff Longitude")
axes[1, 0].set_xlabel("Pickup Longitude")
axes[1, 0].set_ylabel("Dropoff Longitude")
axes[1, 0].grid(True, alpha=0.3)

# Pickup vs Dropoff Latitude
axes[1, 1].scatter(df_cleaned["pickup_latitude"],
                   df_cleaned["dropoff_latitude"],
                   alpha=0.5, s=1, color="orange")
axes[1, 1].set_title("Pickup vs Dropoff Latitude")
axes[1, 1].set_xlabel("Pickup Latitude")
axes[1, 1].set_ylabel("Dropoff Latitude")
axes[1, 1].grid(True, alpha=0.3)

# Distance vs Fare (we'll calculate distance first)
# Calculate Euclidean distance (simplified)
df_cleaned["distance"] = np.sqrt(
    (df_cleaned["dropoff_longitude"] - df_cleaned["pickup_longitude"])**2 +
    (df_cleaned["dropoff_latitude"] - df_cleaned["pickup_latitude"])**2
)
axes[1, 2].scatter(df_cleaned["distance"], df_cleaned["fare_amount"],
                   alpha=0.5, s=1, color="brown")
axes[1, 2].set_title("Distance vs Fare Amount")
axes[1, 2].set_xlabel("Distance (Euclidean)")
axes[1, 2].set_ylabel("Fare Amount ($)")
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("bivariate_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

# 9.3: Statistical Relationship Analysis
print("\n--- Statistical Relationships ---")
from scipy.stats import pearsonr, spearmanr

# Calculate correlations with p-values
relationships = [
    ("fare_amount", "distance"),
    ("fare_amount", "passenger_count"),
    ("pickup_longitude", "dropoff_longitude"),
    ("pickup_latitude", "dropoff_latitude")
]

for var1, var2 in relationships:
    pearson_corr, pearson_p = pearsonr(df_cleaned[var1], df_cleaned[var2])
    spearman_corr, spearman_p = spearmanr(df_cleaned[var1], df_cleaned[var2])
    print(f"\n{var1.upper()} vs {var2.upper()}:")
    print(f" Pearson correlation: {pearson_corr:.4f} (p-value:\n{pearson_p:.4e})")
    print(f" Spearman correlation: {spearman_corr:.4f} (p-value:\n{spearman_p:.4e})") 

In [ ]:
print("=== OUTLIER DETECTION AND ANALYSIS ===")

# 10.1: IQR Method for Outlier Detection
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# 10.2: Z-Score Method for Outlier Detection
def detect_outliers_zscore(data, column, threshold=3):
    z_scores = np.abs((data[column] - data[column].mean()) / data[column].std())
    outliers = data[z_scores > threshold]
    return outliers

# Analyze outliers for fare amounts
print("--- Fare Amount Outliers ---")
fare_outliers_iqr, fare_lower, fare_upper = detect_outliers_iqr(df_cleaned, "fare_amount")
fare_outliers_zscore = detect_outliers_zscore(df_cleaned, "fare_amount")

print(f"IQR Method:")
print(f" Lower bound: ${fare_lower:.2f}")
print(f" Upper bound: ${fare_upper:.2f}")
print(f" Number of outliers: {len(fare_outliers_iqr):,} ({len(fare_outliers_iqr)/len(df_cleaned)*100:.2f}%)")

print(f"\nZ-Score Method (threshold=3):")
print(f" Number of outliers: {len(fare_outliers_zscore):,} ({len(fare_outliers_zscore)/len(df_cleaned)*100:.2f}%)")

# Visualize outliers
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Box plot with outliers
axes[0, 0].boxplot(df_cleaned["fare_amount"], patch_artist=True)
axes[0, 0].set_title("Fare Amount Box Plot (with outliers)")
axes[0, 0].set_ylabel("Fare Amount ($)")
axes[0, 0].grid(True, alpha=0.3)

# Box plot without extreme outliers (for better visualization)
fare_no_extreme = df_cleaned[df_cleaned["fare_amount"] <= df_cleaned["fare_amount"].quantile(0.95)]
axes[0, 1].boxplot(fare_no_extreme["fare_amount"], patch_artist=True)
axes[0, 1].set_title("Fare Amount Box Plot (95th percentile cap)")
axes[0, 1].set_ylabel("Fare Amount ($)")
axes[0, 1].grid(True, alpha=0.3)

# Histogram with outlier boundaries
axes[1, 0].hist(df_cleaned["fare_amount"], bins=100, alpha=0.7, color="skyblue")
axes[1, 0].axvline(fare_lower, color="red", linestyle="--", label=f"Lower\nbound: ${fare_lower:.2f}")
axes[1, 0].axvline(fare_upper, color="red", linestyle="--", label=f"Upper\nbound: ${fare_upper:.2f}")

axes[1, 0].set_title("Fare Amount Distribution with IQR Outlier Bounds")
axes[1, 0].set_xlabel("Fare Amount ($)")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Scatter plot: Distance vs Fare with outliers highlighted
outlier_indices = fare_outliers_iqr.index
normal_data = df_cleaned.drop(outlier_indices)
axes[1, 1].scatter(normal_data["distance"], normal_data["fare_amount"], alpha=0.5, s=1, color="blue", label="Normal")
axes[1, 1].scatter(fare_outliers_iqr["distance"], fare_outliers_iqr["fare_amount"], alpha=0.7, s=2, color="red", label="Outliers")
axes[1, 1].set_title("Distance vs Fare (Outliers Highlighted)")
axes[1, 1].set_xlabel("Distance")
axes[1, 1].set_ylabel("Fare Amount ($)")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("outlier_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

# Analyze characteristics of outliers
print(f"\n--- Outlier Characteristics ---")
if len(fare_outliers_iqr) > 0:
    print("Top 10 highest fare outliers:")
    top_outliers = fare_outliers_iqr.nlargest(10, "fare_amount")[["fare_amount", "distance", "passenger_count", "pickup_datetime"]]
    print(top_outliers)
    print(f"\nOutlier statistics:")
    print(f" Mean fare of outliers:\n${fare_outliers_iqr['fare_amount'].mean():.2f}")
    print(f" Median fare of outliers:\n${fare_outliers_iqr['fare_amount'].median():.2f}")
    print(f" Max fare outlier: ${fare_outliers_iqr['fare_amount'].max():.2f}") 

In [ ]:
print("=== FEATURE ENGINEERING ===")

# Ensure pickup_datetime is in datetime format
df_cleaned["pickup_datetime"] = pd.to_datetime(df_cleaned["pickup_datetime"])

# 11.1: Extracting Time-Based Features
print("Extracting time-based features...")
df_cleaned["year"] = df_cleaned["pickup_datetime"].dt.year
df_cleaned["month"] = df_cleaned["pickup_datetime"].dt.month
df_cleaned["day"] = df_cleaned["pickup_datetime"].dt.day
df_cleaned["hour"] = df_cleaned["pickup_datetime"].dt.hour
df_cleaned["day_of_week"] = df_cleaned["pickup_datetime"].dt.dayofweek # Monday=0, Sunday=6
df_cleaned["day_name"] = df_cleaned["pickup_datetime"].dt.day_name()
df_cleaned["month_name"] = df_cleaned["pickup_datetime"].dt.month_name()

# 11.2: Categorizing Day of Week and Time of Day
print("Categorizing day of week and time of day...")
df_cleaned["is_weekend"] = df_cleaned["day_of_week"].apply(lambda x: 1 if x >= 5 else 0) # Saturday and Sunday

def get_time_of_day(hour):
    if 7 <= hour <= 9 or 16 <= hour <= 19:
        return "Peak"
    elif 22 <= hour or hour <= 5:
        return "Late Night"
    else:
        return "Off-Peak"
df_cleaned["time_of_day"] = df_cleaned["hour"].apply(get_time_of_day)

# 11.3: Calculating Ride Distance (Haversine Distance)
print("Calculating Haversine distance...")
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371 # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

df_cleaned["distance_km"] = df_cleaned.apply(
    lambda row: haversine_distance(
        row["pickup_latitude"], row["pickup_longitude"],
        row["dropoff_latitude"], row["dropoff_longitude"]
    ), axis=1
)

df_cleaned["distance_km"] = df_cleaned["distance_km"].replace([np.inf, -np.inf], np.nan)
df_cleaned["distance_km"].fillna(0, inplace=True) # Replace NaN with 0 for rides with no distance

# 11.4: Fare per Kilometer
print("Calculating fare per kilometer...")
df_cleaned["fare_per_km"] = df_cleaned["fare_amount"] / (df_cleaned["distance_km"] + 0.001) # Add small epsilon to avoid division by zero

# Display new features
print("\nNew features created:")
print(df_cleaned[[
"pickup_datetime", "year", "month", "day", "hour",
"day_of_week", "day_name", "month_name", "is_weekend", "time_of_day",
"distance_km", "fare_per_km"
]].head())

print("\nDescriptive statistics for new numerical features:")
print(df_cleaned[["distance_km", "fare_per_km"]].describe()) 

In [ ]:
print("=== ENCODING CATEGORICAL VARIABLES ===")

# Identify categorical columns for encoding
categorical_cols_to_encode = ["day_name", "month_name", "time_of_day"]

# One-Hot Encoding for nominal categorical variables
# This creates new binary columns for each category
df_encoded = pd.get_dummies(df_cleaned, columns=categorical_cols_to_encode,
                            drop_first=True) # drop_first avoids multicollinearity

print("\nOriginal vs Encoded DataFrame shapes:")
print(f"Original df_cleaned shape: {df_cleaned.shape}")
print(f"Encoded df_encoded shape: {df_encoded.shape}")

print("\nFirst 5 rows of encoded data (showing new columns):")
print(df_encoded.head())

# You can choose to work with df_cleaned (for Power BI) or df_encoded (for ML modeling)
# For the purpose of this guide, we will continue with df_cleaned for Power BI export,
# but keep df_encoded in mind for the modeling section. 

In [ ]:
# Save enhanced dataset
output_enhanced_filename = "uber_enhanced.csv"
df_cleaned.to_csv(output_enhanced_filename, index=False)

print(f"\nEnhanced dataset saved as: {output_enhanced_filename}")
print(f"File size: {len(df_cleaned):,} rows × {len(df_cleaned.columns)} columns")

# Display final enhanced dataset info
print("\n=== FINAL ENHANCED DATASET INFO ===")
print(df_cleaned.info())
print("\nFirst 5 rows of enhanced data:")
print(df_cleaned.head()) 

In [ ]:
print("=== DATASET SPLITTING ===")
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
# We will use the df_encoded for modeling as it contains one-hot encoded
# categorical features
# Drop 'key' and 'pickup_datetime' as they are not direct features for modeling
# Also drop original categorical columns if using one-hot encoded ones
X = df_encoded.drop([
    "fare_amount", "key", "pickup_datetime",
    "day_name", "month_name", "time_of_day" # Drop original categorical columns
], axis=1)
y = df_encoded["fare_amount"]

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

# Split the data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing
# random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

print(f"\nTraining set size: {len(X_train):,} samples")
print(f"Testing set size: {len(X_test):,} samples")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}") 

In [ ]:
print("=== MODEL SELECTION AND TRAINING ===")
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Initialize the Random Forest Regressor model
# n_estimators: number of trees in the forest
# random_state: for reproducibility
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 uses all available cores

print("Training the Random Forest Regressor model...")
# Train the model using the training data
model.fit(X_train, y_train)
print("Model training complete.") 

In [ ]:
print("=== MODEL EVALUATION ===")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n--- Regression Model Performance ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R2): {r2:.4f}")

# Visualize actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.3, s=10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         linestyle="--", color="red", linewidth=2, label="Perfect Prediction")
plt.title("Actual vs. Predicted Fare Amounts", fontsize=16,
          fontweight="bold")
plt.xlabel("Actual Fare Amount ($)")
plt.ylabel("Predicted Fare Amount ($)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.savefig("actual_vs_predicted.png", dpi=300, bbox_inches="tight")
plt.show()

# Visualize residuals
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=50, kde=True, color="skyblue")
plt.title("Distribution of Residuals", fontsize=16, fontweight="bold")
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)
plt.savefig("residuals_distribution.png", dpi=300, bbox_inches="tight")
plt.show()

# Feature Importance (for tree-based models)
print("\n--- Feature Importance ---")
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances.values, y=feature_importances.index,
            palette="viridis")
plt.title("Feature Importances from Random Forest Regressor", fontsize=16,
          fontweight="bold")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=300, bbox_inches="tight")
plt.show()

print("Top 10 most important features:")
print(feature_importances.head(10)) 